## duplicate fixing
I have discovered I have more images in my file than in images - algortithm has picked up more particles than originally sliced. so images have probably been overwriten (I should have originally accounted for this). none the less this has been predominantly fixed


In [1]:
import numpy as np
import pandas as pd
import os
#from PIL import Image
from glob import glob
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import shutil

from numba import jit
import h5py

In [2]:
duplicate_loc = '/gws/nopw/j04/dcmex/users/ezriab/silly_duplicates/'
path_2ds_stats = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds.csv'
all_2ds_dup_csv = 'duplicates_df.csv'
all_2ds_dup_fix_csv = 'full_dup_fixed_names.csv'
full_ds_stats = pd.read_csv(path_2ds_stats) ## all 2ds images extracted + with corresponding stats
all_2ds_dup = pd.read_csv(duplicate_loc+all_2ds_dup_csv) ## original file with all duplicates 
all_2ds_dup_fixed = pd.read_csv(duplicate_loc+all_2ds_dup_fix_csv) ## all duplicate images, with updated name
# new name of image corresponds to the ones in the 'silly_duplicates' folder

In [3]:
## newly ran + obtained list of images - with new stats associated with the particle
# now it looks like i will have to get rid of a few particles
dup_3_csv = 'updated_duplicates_3nd.csv'
dup_3_df = pd.read_csv(duplicate_loc+dup_3_csv)

all_2ds_dup_fixed

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
0,94298_1_19ch0,220719,94298,94332,53:05.0,53:05.0,349.710133,392.045916,402.243707,71300,...,25,0,0,1.219161,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
1,94298_2_19ch0,220719,94298,94311,53:05.0,53:05.0,146.839578,130.000000,140.356688,4400,...,0,0,0,3.089049,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
2,226470_1_19ch0,220719,226470,226482,56:50.7,56:50.7,152.272582,126.491106,143.178211,4200,...,0,0,0,3.395385,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
3,226470_3_19ch0,220719,226470,226483,56:50.7,56:50.7,174.590712,172.046505,186.010752,13000,...,0,0,0,1.533949,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
4,230869_1_19ch0,220719,230869,230879,56:50.8,56:50.8,135.556367,144.222051,152.643375,8000,...,0,0,0,1.263412,-2.762317,388.69946,7885.3310,255.70236,0.186962,0.442953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,315316_7_07ch1,220807,315316,315356,34:41.3,34:41.3,417.369190,512.249939,520.096145,88900,...,0,33,0,1.315120,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000
291,18539_1_07ch1_!,220807,18539,18595,18:02.1,18:02.1,579.097229,676.239603,688.186021,146400,...,6,0,0,1.589710,0.090431,431.38864,7046.8150,260.83000,0.000063,0.000000
292,34578_1_07ch1_!,220807,34578,34691,20:07.1,20:40.1,977.017933,1136.353818,1146.298390,223500,...,0,83,0,3.120222,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012
293,34578_3_07ch1,220807,34578,34603,20:07.1,20:40.1,258.089120,259.615100,269.258240,8200,...,0,17,0,4.987685,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012


In [4]:
# = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample'
sample_labelled_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd.csv'
image_labelled_df = pd.read_csv(sample_labelled_path)
images_labelled_lst = list(image_labelled_df['image_name'])
labelled_image_stats = full_ds_stats[full_ds_stats['name'].isin(images_labelled_lst)] # extract the ones labelled

# combine the stats with the image labels
image_labelled_df = image_labelled_df.rename(columns={'image_name': 'name'}) # need to rename our column, so they merge nicely
merged_df = pd.merge(labelled_image_stats, image_labelled_df, on='name', how='left')  # merge them together

# doing duplicate work
full_duplicates_mask = merged_df.duplicated(subset=['name'])
full_duplicates_df = merged_df[full_duplicates_mask]

list_of_duplicates = list(full_duplicates_df['name'])
double_duplicates_df = full_ds_stats[full_ds_stats['name'].isin(list_of_duplicates)]
## ^ these are the ones that are duplicated in the original sample 
double_duplicates_df

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
855824,2484896_25ch1,220725,2484896,2484919,19:37:49.125000000,19:37:49.125000000,331.417465,317.804972,331.209903,43200.0,...,0,0,0,1.575712,-0.382931,380.90970,7979.3060,256.33566,0.006337,0.284747
855825,2484896_25ch1,220725,2484896,2484914,19:37:49.125000000,19:37:49.125000000,217.905812,184.390889,196.468827,5000.0,...,0,0,0,3.949269,-0.382931,380.90970,7979.3060,256.33566,0.006337,0.284747
923312,47102_07ch1,220807,47102,47126,16:37:00.357000000,16:37:03.899000000,273.403521,282.842712,290.688837,41600.0,...,18,0,0,1.283139,0.561548,446.08530,6787.3706,261.83868,0.001382,0.000019
940222,47102_07ch1,220807,47102,47131,19:26:38.926000000,19:26:39.519000000,289.030963,318.904374,328.024389,50300.0,...,0,0,0,1.172358,-4.348890,410.21915,7428.8936,257.82507,0.204330,0.008787


### fixing time
i have not been paying attention, but the time has gone wrong when i have been downloading it and manually changing the lables. 
i need to fix this.

### fixing the csvs that alreay exist?

In [23]:
## some of the ones i have fixed still dont match up correctly / names have been repeated so duplicated again
mask_weird_images = all_2ds_dup_fixed.name.apply(lambda x: '!' in x) ## ive put '!' in the names of ones i know are not good
unusable_particles = all_2ds_dup_fixed[mask_weird_images] ## will save this to the stats file, so i am aware of which ones I have removed

not_viable_particles = ['745_07ch0', '31047_07ch0','18539_07ch1','47102_07ch1', '34578_07ch1']

#save_unusable = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/removed_duplicates.csv'
#unusable_particles.to_csv(save_unusable, index=False) # the ones that couldnt be saved are here       

unusable_particles_lst = list(unusable_particles['name']) 
## fixing the large 2ds file, containing all the info
bad_dup_inx = []
for bad_duplicate in unusable_particles_lst: 
    bad_dup_inx.append(all_2ds_dup_fixed.index[all_2ds_dup_fixed['name'] == bad_duplicate])
hash_bad_dup_idx = list(set([idx for index in bad_dup_inx for idx in index])) # this makes the list of indexs iterable

bad_2ds_dup_dropped = all_2ds_dup_fixed.drop(index = (hash_bad_dup_idx)) 
bad_2ds_dup_dropped # left with the newly labelled particles, with usable images 

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
0,94298_1_19ch0,220719,94298,94332,53:05.0,53:05.0,349.710133,392.045916,402.243707,71300,...,25,0,0,1.219161,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
1,94298_2_19ch0,220719,94298,94311,53:05.0,53:05.0,146.839578,130.000000,140.356688,4400,...,0,0,0,3.089049,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
2,226470_1_19ch0,220719,226470,226482,56:50.7,56:50.7,152.272582,126.491106,143.178211,4200,...,0,0,0,3.395385,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
3,226470_3_19ch0,220719,226470,226483,56:50.7,56:50.7,174.590712,172.046505,186.010752,13000,...,0,0,0,1.533949,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
4,230869_1_19ch0,220719,230869,230879,56:50.8,56:50.8,135.556367,144.222051,152.643375,8000,...,0,0,0,1.263412,-2.762317,388.69946,7885.3310,255.70236,0.186962,0.442953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,291817_1_07ch1,220807,291817,291856,34:37.2,34:37.2,428.693711,452.769257,460.108683,55000,...,0,0,0,2.222421,-4.712656,416.81490,7306.7656,258.59982,0.343610,0.134197
288,291817_8_07ch1,220807,291817,291827,34:37.2,34:37.2,102.189278,104.403065,117.046999,4800,...,0,0,0,1.412067,-4.712656,416.81490,7306.7656,258.59982,0.343610,0.134197
289,315316_6_07ch1,220807,315316,315328,34:41.3,34:41.3,138.631651,123.693169,136.014705,4000,...,0,0,0,3.264619,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000
290,315316_7_07ch1,220807,315316,315356,34:41.3,34:41.3,417.369190,512.249939,520.096145,88900,...,0,33,0,1.315120,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000


In [24]:
## now we want to remove all duplicates from the big 2ds csv
ds_dup = list(all_2ds_dup['name'])

# get list of indexs that correspond to duplicates
dup_inx = []
for duplicate in ds_dup: 
    dup_inx.append(full_ds_stats.index[full_ds_stats['name'] == duplicate])

full_ds_stats.iloc[dup_inx[0]]
hash_dup_idx = list(set([idx for index in dup_inx for idx in index])) # this makes the list of indexs iterable

dup_drop_full_ds_stats = full_ds_stats.drop(index = (hash_dup_idx)) # this removes the duplicates with incorrect names
print(full_ds_stats['name'].count())
print(dup_drop_full_ds_stats['name'].count())

KeyboardInterrupt: 

In [37]:
# now concat original file, with the correctly updated ones
fixed_dup_all_2ds = pd.concat([dup_drop_full_ds_stats, bad_2ds_dup_dropped], ignore_index=True)
save_fixed = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds_dup_updated.csv'
#fixed_dup_all_2ds.to_csv(save_fixed, index=False)
#fixed_dup_all_2ds


In [29]:
## repeating this process for the ones i have sub-sampled 
# and also needing to move the corresponding images to this file
# then re-download the sample + label them correctly

sample_dup = list(double_duplicates_df['name'])
samp_dup_inx = []
for duplicate in sample_dup: 
    samp_dup_inx.append(image_labelled_df.index[image_labelled_df['name'] == duplicate])

hash_bad_dup_samp_idx = list(set([idx for index in samp_dup_inx for idx in index])) # this makes the list of indexs iterable
hash_bad_dup_samp_idx
first_dup = image_labelled_df.iloc[hash_bad_dup_samp_idx[0]]
second_dup = image_labelled_df.iloc[hash_bad_dup_samp_idx[1]] ## we are going to have to just get rid of this one

## when looking at the images, i think i have supposed to have labelled 2484896_1_25ch1, which i will now fix - this also need a new lable
## but i will be getting rid of 47102_07ch1

dup_drop_labelled = image_labelled_df.drop(index = (hash_bad_dup_samp_idx)) # removing duplicates
new_row = pd.DataFrame({'name':'2484896_1_25ch1','number_label':0,'note': np.nan}, index=[0])

updated_labelled_df = pd.concat([dup_drop_labelled, new_row], ignore_index=True)
updated_labelled_df ## newly fixed df, with correct label

save_image_label = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd_fixed_dup.csv'
#updated_labelled_df.to_csv(save_image_label, index=False)

In [31]:
## need to copy the newly updated image to the sample folder
start_loc = '/gws/nopw/j04/dcmex/users/ezriab/silly_duplicates/2484896_1_25ch1.png'
end_loc = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
#shutil.copy(start_loc, end_loc)    

'/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2484896_1_25ch1.png'

### fixing the original location of the images
now the big csv file is fixed, need to add the newly updated images to their respective folders

In [6]:
## actually, first of all i am going to combine the days together
# some days h5 files have been seperated out
file_days = pd.unique(full_ds_stats['date'])
file_days

array([220719, 220720, 220722, 220723, 220725, 220726, 220727, 220729,
       220730, 220801, 220802, 220803, 220804, 220806, 220807, 220808])

### fix code ran
#### obtained images in silly_duplicates folder

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import pandas as pd
from scipy.special import gamma
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
import os
import psutil

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from scipy.spatial.distance import pdist
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
import tensorflow as tf

from numba import jit

In [13]:
## og functions / things to make script run smoothly
## setting thresholds / res for attaining good particle final images
fill_hole_threshold = 5 # max number pixels contained within particle that is filled in

minimum_area = 20 # very quick metric to stop the processing of particles with area < 15 pixels
desired_image_size = 200 
pixel_resolution = 10 # mu for 2DS // need to be 150 for hvps
length_threshold = 100 ## 100 mu for 2ds // 1500 mu for hvps

def stats_description(bw_crystal, fill_hole_thresh):
    #take binary image, fill in small holes and returns object containing stats about crystal
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=fill_hole_thresh) # fill in voids within binary image - better estimation of stats # may need to be altered
       
    if filled_particle.shape[0] < 2 or filled_particle.shape[1] < 2:
        return filled_particle, None
        
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return filled_particle, region
    else:
        return filled_particle, None


## function to calculate truncation of particle
@jit(nopython=True) # Enables full optimization by numba
def calc_truncation(particle_coords):
    ## so much simpler, looking at list of coordinates making up a particle, then summing ones in 0 and 127 row - i.e. first + last diode
    lst_first_diode = [coord for coord in particle_coords if coord[0] == 0]
    lst_last_diode = [coord for coord in particle_coords if coord[0] == 127]

    n_top = len(lst_first_diode)
    n_bottom = len(lst_last_diode)

    return n_top, n_bottom # number pixels touching top / bottom respectively
## 

###  set up dataframe, used to extract from raw h5 file + has stats about the particle
columns = [
    "name",
    "date",
    "slice_s_idx",
    "slice_e_idx",
    #"start_time", #hh:mm:ss 
    #"end_time", #hh:mm:ss 
    "ellipse_d_max", # um
    #"ellipse_d_min", # um
    "Euclidean_d_max", # um
    "Feret_d_max", # um
    "area", # um2
    "perimeter", # um
    "circularity",
    "probe",
    "first_diode_trunc",
    "last_diode_trunc",
    "image_trunc",
    "aspect_ratio"
    ]

In [12]:
## second go - locate name pairs, then use the max end index 
#dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv'
dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/duplicates_df.csv'
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/do_it_again/'

base_h5_pth = '/gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/'
particle_df = pd.DataFrame(columns=columns)

######## NEW ###########
dup_df = pd.read_csv(dup_csv_pth)
duplicates_mask = dup_df.duplicated(subset=['name'])
one_dup_df = dup_df[duplicates_mask]
dup_name_lst = list(one_dup_df['name']) # list of 1 name of all the duplicates
######################

'''
with open(dup_csv_pth, mode='r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Read the header row
    # loop through each row
    for row in csv_reader:
        image_name = row[0]
        date = row[1] 
        start_idx = row[2]
        end_idx = row[3]
        probe = row[12]
        date_day = date[-2:]
'''
    ######## NEW ###########
for i in range(len(dup_name_lst)):
    row = dup_df[dup_df['name'] == dup_name_lst[i]] # all rows with same name
    image_name = max(row['name'].values)
    date = str(max(row['date'].values))
    start_idx = max(row['slice_s_idx'].values)
    end_idx = max(row['slice_e_idx'].values)
    probe = max(row['probe'].values)
    date_day = date[-2:]

    ########################
    print(image_name)    
    h5_of_interest = [] ## some have multiple files associated with each date
    list_h5_files = glob(base_h5_pth+probe+'/'+'*.h5')
    for file in list_h5_files:
        if date in file:
            h5_of_interest.append(file)

    for h5_path in h5_of_interest:
        h5_file = h5py.File(h5_path,'r')

        try:
            h5_image = h5_file['ImageData']
            h5_time = h5_file['ImageTimes']
        except KeyError as e:
            print(f"Dataset missing in file: {file}. Error: {e}")
            continue

        one_crystal = h5_file['ImageData'][:,int(start_idx):int(end_idx)] # extract 1 crystal
        #plt.imshow(one_crystal, cmap='gray')
        binary_image = (one_crystal == 0) 
        labeled_image, num_features = label(binary_image)
        try:
            props = regionprops(labeled_image)
        except ValueError as e:
            print(f' {file} at start idx {start_idx}, something has gone wrong')
            continue
            
        for particle in props:
            if particle.area >= minimum_area:
                filled_part, spec_region = stats_description(particle,fill_hole_threshold)
                if spec_region:
                    aspect_ratio_value = spec_region.major_axis_length / spec_region.minor_axis_length
                    coords = particle.coords # basically gives coords of each point of interest [row,column]
                    distances = pdist(coords)
                    euclidean_dim = np.max(distances)

                    if euclidean_dim * pixel_resolution >= length_threshold:
                            
                        ## basic info
                        x_values = np.unique(coords[:, 1])
                        e_idx = start_idx+x_values[-1]
                        #s_idx = int(selected_pix_sum[i] + x_values[0])
                        #e_idx = int(selected_pix_sum[i] + x_values[-1])
        
                        ## truncation calc
                        # cnn - in final pic size, particle may be truncated if very long, this tells us how many pixels may be cut off 
                        image_trunc = x_values[-1] - desired_image_size 
                        if image_trunc < 0:
                            image_trunc = 0
                        # normal trunc calculation - on actual probe
                        first_diode, last_diode = calc_truncation(coords)
                        
                        ## using circularity calculation from Crosier et al. 2011
                        circularity_calc = np.divide((spec_region.perimeter**2),(4*np.pi*spec_region.area))
                        particle_name = f'{start_idx}_{particle.label}_{date_day}{probe}'
                        
                        # nice way of saving data - lenth + measurements are correct in microns
                        one_particle_data = {
                                #"image_index": image_index,
                                "name": particle_name,
                                "date" : date,
                                "slice_s_idx": start_idx,
                                "slice_e_idx": e_idx,
                                #"start_time": str(selected_utc_time[i].values).split('T')[1], # more friendly time
                                #"end_time": str(selected_utc_time[i+1].values).split('T')[1], # more friendly time
                                "ellipse_d_max": spec_region.major_axis_length * pixel_resolution, ## d_max (equivalent ellipse)
                                #"ellipse_d_min": spec_region.minor_axis_length * pixel_resolution, ## d_min (equivalent ellipse)
                                "Euclidean_d_max": euclidean_dim * pixel_resolution,
                                "Feret_d_max":spec_region.feret_diameter_max * pixel_resolution,
                                "area": (spec_region.area * (pixel_resolution**2)),
                                "perimeter": (spec_region.perimeter * pixel_resolution),
                                "circularity": circularity_calc,
                                "probe": probe,
                                "first_diode_trunc": first_diode,
                                "last_diode_trunc": last_diode,
                                "image_trunc": image_trunc,
                                "aspect_ratio": aspect_ratio_value  
                                }

                        filled_part = filled_part.astype(np.float32) ## convert to float 0 and 1s
                        filled_part = np.expand_dims(filled_part, axis=-1) ## add extra dimention - this is for adding padding
        
                        imagex = tf.image.resize_with_crop_or_pad(filled_part, desired_image_size, desired_image_size)
                        ## save image
                        # Remove the extra dimension if needed
                        image_np = imagex.numpy().squeeze()
                        #plt.imshow(image_np, cmap='gray')
                        plt.imsave(f'{save_loc}{particle_name}.png', image_np, cmap="gray")
                        print(f'{particle_name} done')

                        one_particle_data_df = pd.DataFrame([one_particle_data])
                        
                        particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)
        print(f'{image_name} done')
        h5_file.close()
        #print(one_particle_data)

particle_df.to_csv(f'{save_loc}updated_duplicates_3nd.csv', index=False)        
        

94298_19ch0
94298_1_19ch0 done
94298_2_19ch0 done
94298_19ch0 done
226470_19ch0


/tmp/ipykernel_7116/2537520632.py:132: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)


226470_1_19ch0 done
226470_3_19ch0 done
226470_19ch0 done
230869_19ch0
230869_1_19ch0 done
230869_4_19ch0 done
230869_19ch0 done
235921_19ch0
235921_1_19ch0 done
235921_3_19ch0 done
235921_19ch0 done
249985_19ch0
249985_1_19ch0 done
249985_2_19ch0 done
249985_19ch0 done
364115_19ch0
364115_1_19ch0 done
364115_13_19ch0 done
364115_19ch0 done
806361_19ch0
806361_1_19ch0 done
806361_2_19ch0 done
806361_19ch0 done
830185_19ch0
830185_1_19ch0 done
830185_2_19ch0 done
830185_19ch0 done
1188953_19ch0
1188953_1_19ch0 done
1188953_2_19ch0 done
1188953_19ch0 done
1403973_19ch0
1403973_1_19ch0 done
1403973_2_19ch0 done
1403973_19ch0 done
14000_20ch0
14000_1_20ch0 done
14000_5_20ch0 done
14000_20ch0 done
367051_20ch0
367051_1_20ch0 done
367051_3_20ch0 done
367051_20ch0 done
568208_20ch0
568208_1_20ch0 done
568208_3_20ch0 done
568208_20ch0 done
728277_20ch0
728277_1_20ch0 done
728277_2_20ch0 done
728277_20ch0 done
1003345_20ch0
1003345_1_20ch0 done
1003345_2_20ch0 done
1003345_20ch0 done
1016899_20